In [1]:
%load_ext zipline

注意以下测试的期间。财务报告默认公告日期在报告期后45天。11月14日查询到的数据是二季度报告；<span class="girk">11月15日后，才是三季度报告。</span>

In [2]:
%%zipline --start 2017-11-1 --end 2017-11-16 --capital-base 100000 --bm_symbol 000001

from zipline.pipeline import Pipeline, Fundamentals
from zipline.pipeline.factors import CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.api import attach_pipeline, pipeline_output

from zipline.api import symbol, sid, get_datetime
from numpy.testing import assert_almost_equal
import pandas as pd

check_dates = [pd.Timestamp('2017-11-14'), pd.Timestamp('2017-11-15')]
check_stocks = ['600050', '000001']
b001 = [[13815993, 20577758],[5407300, 7983300]]
b044 = [[0.04, 0.06],[0.68, 1.06]]

def check_data(today, output, idx, col_name, col_data):
    check_date = check_dates[idx].date()
    if today == check_date:
        stock = check_stocks[idx]
        actual = output.loc[symbol(stock), col_name]
        desired = col_data[idx][idx]
        if assert_almost_equal(actual, desired) is None:
            print('股票 {} 日期 {} 项目 {} 通过测试'.format(stock, check_date, col_name))

def make_pipeline():
    return Pipeline(
        columns = {
            'asof_date':Fundamentals.income_statement.asof_date.latest,
            '营业总收入(万元)':Fundamentals.income_statement.B001.latest,  
            '基本每股收益':Fundamentals.income_statement.B044.latest,              
        },
    )

def initialize(context):      
    attach_pipeline(make_pipeline(), 'example')

def handle_data(context, data):
    today = get_datetime('Asia/Shanghai').tz_localize(None).date()
    output = pipeline_output('example')
    check_data(today, output, 0, '营业总收入(万元)', b001)
    check_data(today, output, 0, '基本每股收益', b044)    
    
    check_data(today, output, 1, '营业总收入(万元)', b001)
    check_data(today, output, 1, '基本每股收益', b044)      

ValueError: no data for bundle 'cnstock' on or before 2018-01-05 03:10:34.178744+00:00
maybe you need to run: $ zipline ingest -b cnstock